In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from PyEMD import EMD
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

In [ ]:
path = "D:\BaiduNetdiskDownload\data\\donghai\\2011-2014.nc"
dataset = xr.open_dataset(path)

In [ ]:
target_longitude_b = 120.00
target_latitude_b = 39.00
subset_data = dataset.sel(longitude=target_longitude_b, latitude=target_latitude_b, method='nearest')

In [ ]:
time = subset_data.variables['time'][:].data
u10 = subset_data.variables['u10'][:].data
v10 = subset_data.variables['v10'][:].data
swh = subset_data.variables['swh'][:].data
# 创建DataFrame来组织数据
data = pd.DataFrame({'time': time, 'u10': u10, 'v10': v10, 'swh': swh})

In [ ]:
# length = len(swh)
# time = np.linspace(0, length, length)
# emd = EMD()
# IMFs = emd(swh)

# plt.figure(figsize=(10, 5))
# plt.subplot(len(IMFs) + 1, 1, 1)
# plt.plot(time, swh, 'r')
# plt.title("Original SWH")

# for i, imf in enumerate(IMFs):
#     plt.subplot(len(IMFs) + 1, 1, i + 2)
#     plt.plot(time, imf)
#     plt.title(f"IMF {i + 1}")
# plt.xlabel('Time (h)')
# plt.tight_layout()
# plt.savefig('D_emd.png')
# plt.show()

#######################################
time = np.linspace(0, 15000, 15000)
emd = EMD()
IMFs = emd(swh[:15000])

plt.figure(figsize=(5, 10))
plt.subplot(len(IMFs) + 1, 1, 1)
plt.plot(time, swh[:15000], 'r')
plt.title("Original SWH")

for i, imf in enumerate(IMFs):
    plt.subplot(len(IMFs) + 1, 1, i + 2)
    plt.plot(time, imf)
    plt.title(f"IMF {i + 1}")
plt.xlabel('Time (h)')
plt.tight_layout()
plt.savefig('D_emd.png')
plt.show()
########################################


In [ ]:
import pickle
with open('D_EMD_2011-2014.pkl', 'wb') as file:
    pickle.dump(IMFs, file)

In [ ]:
X = np.vstack(IMFs)
X = X.T
y = swh[:15000]
X = np.concatenate((X, data[['u10', 'v10']].values[:15000]), axis=1)
print(X.shape)

In [ ]:
# 划分数据集为训练集和测试集
split_point = int(0.8 * len(X))
X_train = X[:split_point,:]
X_test = X[split_point:,:]
y_train = y[:split_point]
y_test = y[split_point:]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 标准化特征
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
# 将数据转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
print(X_train_tensor.shape)
print(y_train_tensor.shape)

In [ ]:
# 创建PyTorch数据集和数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out

In [ ]:
# 创建模型实例
input_size = X.shape[1]
hidden_size = 50
model = LSTMModel(input_size, hidden_size)

In [ ]:
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [15]:
# 训练模型
num_epochs = 20
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.unsqueeze(1))
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# 测试模型
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor.unsqueeze(1))
    test_loss = criterion(y_pred_tensor, y_test_tensor.unsqueeze(1))

print(f'Test Loss: {test_loss.item():.4f}')

In [ ]:

y_pred = y_pred_tensor.numpy()
y_test = y_test_tensor.numpy()


In [ ]:
swh_pred = np.sum(y_pred.T, axis=0)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test, swh_pred))
print("RMSE:", rmse)


from scipy.stats import pearsonr
r, _ = pearsonr(y_test, swh_pred)
print("Correlation coefficient (R):", r)

In [ ]:
import matplotlib.pyplot as plt

time = np.arange(0, 400, 1)

#print(y_test[:400])

plt.figure(figsize=(6, 2))
plt.plot(time, y_test[800:1200], linestyle='-', linewidth=1.0)
plt.plot(time, swh_pred[800:1200], linestyle='-', linewidth=1.0)

plt.xlabel('time(h)')
plt.ylabel('height(m)')
plt.title('plot_height')

plt.grid(True)

plt.show()